This is a document I use to separate out functions of the main script in order to troubleshoot or quickly bypass some steps. I often use this to avoid spamming chat or API requests when I want to test changes in a script without using quota credits or some other resource.

In [1]:
from funcs import *
import requests,time
showbot_channel,token = 'Frogpants','Nhrj6amcz4AqiSP6AVv5YhhQX8OhJ6wO'

In [2]:
streamid = find_stream_id('@ScottJohnson')
if streamid:
    print(f'Live stream detected.\nLive stream id: {streamid}')
else:
    print('No live stream found.')

Live stream detected.
Live stream id: uTXnAfFGo6U


In [3]:
streamid = 'TqYpYY4ooSY'

For testing

In [4]:
authors,messages = parse_chat(read_chat(find_chat_id(streamid)))
for author,message in zip(authors,messages):
    print(f'{author}: {message}')

For titles

In [17]:
author_index,titles = title_search(read_chat(find_chat_id(streamid)))
# for author,title in zip(authors,titles):
#     print(f'{author}: {title}')

In [18]:
authors_html,titles_html = [html_ify(auth) for auth in author_index],[html_ify(title) for title in titles]
for author_html,title_html in zip(authors_html,titles_html):
    print(f'{author_html}: {title_html}')
    

In [4]:
livechatid = find_chat_id(streamid)

In [6]:
livechatid = 'Cg0KC3VUWG5BZkZHbzZVKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdVRYbkFmRkdvNlU'

In [7]:
author_index,submitted_titles = build_submission_history(showbot_channel)

In [21]:
for author,title in zip(authors,titles):
    print(f'{author}: {title}')

In [ ]:
while True:
    authors,titles = title_search(read_chat(livechatid)) #Poll API for all chat messages, regardless of they've been polled already or not. title_search() already filters for '!s' chat trigger and cleans as needed
    #Check extracted titles versus previously submitted ones, submit if new
    for author,title in zip(authors,titles):
        if title not in submitted_titles: #Check extracted titles versus previously submitted ones, submit if new
            author_index.append(author)
            submitted_titles.append(title)
            print(f'{author}: {title}') #Add new title to list and add author to another list to maintain the order. Print for debugging and monitoring purposes.
            author_html,title_html = html_ify(author),html_ify(title) #Prepare link for submission then submit via Requests module
            submission_link = f'http://www.showbot.tv/s/add.php?title={title_html}&user={author_html}&channel={showbot_channel}&key={token}'
            requests.get(submission_link)
            #At this point the title has been submitted. This is one complete function and API call. Confirmation via the .insert() method to send a message is a separate 
            #function incurring higher quota useage.
            credentials = build_credentials(client_secrets_file,scopes) #Builds OAuth2 credentials object. Uses existing files or makes new one if not detected. This links the function to the account that follows the URI.
            youtube = build_yt_obj(googleapiclient.discovery,credentials) #Common step in calling the API. Always have to build an object on which to call methods.
            response = send_chat_message(livechatid,author) #The API response in JSON format. Not technically needed but saved in case.
            print(f'Title successfully submitted and confirmed: {author}') #Generic success message only for the console for debugging and monitoring purposes.
    time.sleep(5)